# Stáhnutí meteodat pro stanici Praha-Ruzyně a další

In [1]:
from datetime import datetime

import pandas as pd
from meteostat import Hourly, Stations, Daily

In [3]:
# Najděme nejbližší stanice k Praze
lat, lon = 50.0755, 14.4378  # Praha
s = Stations()
s = s.nearby(
    lat=lat,
    lon=lon,
    radius=250000  # v m?
)
stations = s.fetch(1000)
stations   # pandas DataFrame

,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end,distance
id,,,,,,,,,,,,,,,,
11520,Praha-Libus,CZ,PR,11520,<NA>,50.0167,14.4500,304.0,Europe/Prague,NaT,NaT,1973-01-01,2025-08-24,1972-01-01,2021-01-01,6596.042516
11567,Praha Kbely,CZ,PR,11567,<NA>,50.1167,14.5333,286.0,Europe/Prague,2022-04-19,2022-04-23,2022-04-20,2022-04-22,NaT,NaT,8209.353634
11518,Praha / Ruzyne,CZ,ST,11518,LKPR,50.1000,14.2500,365.0,Europe/Prague,1929-08-02,2025-09-26,1940-01-01,2025-08-24,1940-01-01,2022-01-01,13672.589034
LKVO0,Vodochody,CZ,ST,<NA>,LKVO,50.2166,14.3958,280.0,Europe/Prague,2020-01-14,2025-09-23,NaT,NaT,NaT,NaT,15972.493875
11509,Doksany,CZ,US,11509,<NA>,50.4667,14.1667,158.0,Europe/Prague,NaT,NaT,NaT,NaT,NaT,NaT,47575.455792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CUU0V,Bleßberg,DE,TH,<NA>,<NA>,50.4465,11.0060,865.0,Europe/Berlin,2021-02-04,2025-09-24,NaT,NaT,NaT,NaT,247392.742882
D2541,Katzhütte,DE,TH,<NA>,<NA>,50.5467,11.0317,452.0,Europe/Berlin,2002-11-01,2025-09-24,2004-05-13,2022-04-26,2004-01-01,2022-01-01,247456.895576
AUV78,Altendorf/oberfranken,DE,BY,<NA>,<NA>,49.7978,10.9998,250.0,Europe/Berlin,2003-04-27,2025-09-24,NaT,NaT,NaT,NaT,247961.046368


In [4]:
# Použijeme české stanice s rozumnými daty (Klementinum chybí)
selected_stations = stations[(stations["hourly_start"] < pd.to_datetime("1980-01-01")) & (stations["country"] == "CZ")]
selected_stations

,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end,distance
id,,,,,,,,,,,,,,,,
11518,Praha / Ruzyne,CZ,ST,11518,LKPR,50.1000,14.2500,365.0,Europe/Prague,1929-08-02,2025-09-26,1940-01-01,2025-08-24,1940-01-01,2022-01-01,13672.589034
11448,Plzen-Line,CZ,PL,11448,LKLN,49.6700,13.2700,364.0,Europe/Prague,1939-04-01,2025-09-26,1997-03-01,2022-04-24,2010-01-01,2022-01-01,95061.053714
11414,Karlovy Vary,CZ,KA,11414,LKKV,50.2000,12.9167,603.0,Europe/Prague,1939-01-01,2025-09-26,1998-12-24,2022-04-25,2000-01-01,2022-01-01,109286.757268
11723,Brno / Turany,CZ,JM,11723,LKTB,49.1500,16.7000,241.0,Europe/Prague,1939-03-20,2025-09-26,1939-04-01,2025-08-24,1939-01-01,2022-01-01,192746.016585
11774,Holesov,CZ,ZL,11774,LKHO,49.3167,17.5667,224.0,Europe/Prague,1953-04-04,2025-09-26,1998-10-31,2022-04-24,2005-01-01,2022-01-01,240321.514177


In [5]:
def get_hourly_data(station_id, start) -> pd.DataFrame:
    source = Hourly(station_id, start=pd.to_datetime(start), end=datetime(2025, 8, 31, 23, 59, 59))
    return source.fetch().reset_index()

def get_daily_data(station_id, start) -> pd.DataFrame:
    source = Daily(station_id, start=pd.to_datetime(start), end=datetime(2025, 8, 31, 23, 59, 59))
    return source.fetch().reset_index()

Denní data pro pět českých stanic do jednoho excel souboru:

In [ ]:
with pd.ExcelWriter("data-daily.xlsx") as writer:
    for station_id, station in selected_stations.iterrows():
        df = get_daily_data(station_id, station["daily_start"])
        df.to_excel(writer, sheet_name=station["name"].replace(" / ", "-"), index=False)  # název listu max 31 znaků

Data po hodině z Ruzyně:

In [7]:
prague_hourly = get_hourly_data(11518, "1929-08-02")
prague_hourly.to_parquet("./prague-meteostat.parquet")
prague_hourly

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,1931-01-03 12:00:00,0.0,<NA>,<NA>,<NA>,<NA>,0.0,0.0,<NA>,1004.3,<NA>,<NA>
1,1931-01-03 18:00:00,1.1,<NA>,<NA>,<NA>,<NA>,0.0,0.0,<NA>,1002.8,<NA>,<NA>
2,1931-01-04 06:00:00,6.1,<NA>,<NA>,<NA>,<NA>,200.0,16.6,<NA>,1000.7,<NA>,<NA>
3,1931-01-04 12:00:00,11.1,<NA>,<NA>,<NA>,<NA>,160.0,3.6,<NA>,999.0,<NA>,<NA>
4,1931-01-04 18:00:00,6.1,<NA>,<NA>,<NA>,<NA>,270.0,33.5,<NA>,1001.8,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...
471611,2025-08-31 19:00:00,19.4,9.3,52.0,0.0,<NA>,100.0,14.0,25.9,1012.3,0.0,1.0
471612,2025-08-31 20:00:00,17.7,9.1,57.0,0.0,<NA>,120.0,14.0,29.6,1012.5,0.0,1.0
471613,2025-08-31 21:00:00,17.1,9.0,59.0,0.0,<NA>,130.0,14.0,29.6,1012.3,0.0,1.0
471614,2025-08-31 22:00:00,16.9,9.3,61.0,0.0,<NA>,140.0,14.0,29.6,1012.1,0.0,1.0
